In [1]:
from dgl.data.utils import load_graphs
import dgl
import pandas as pd

graphs, graph_labels = load_graphs('/home/hdd/qingao/graphs.bin')
graphs_by_id = dict(zip(graph_labels["graph_ids"].tolist(), graphs))

graph_ids = graph_labels["graph_ids"].tolist()
df = pd.DataFrame(graph_ids, columns=["graph_id"])

# 保存为 valid_index.csv
df.to_csv("valid_index.csv", index=False)

print("graph_ids 已成功保存为 valid_index.csv")
len(df)


graph_ids 已成功保存为 valid_index.csv


9333

In [2]:
num_nodes_list = [g.number_of_nodes() for g in graphs]

# 计算平均结点数目
average_num_nodes = sum(num_nodes_list) / len(num_nodes_list)

print(f"所有图的平均结点数目: {average_num_nodes}")


所有图的平均结点数目: 201.24472302582234


In [3]:
import pandas as pd
import torch as th
from tqdm import tqdm

allfeats = [
    "api", "datatype", "literal", "operator",
]

def load_additional_features(graphs_by_id, feat_names, split="fixed", sample_text=""):
    """
    加载并将特征添加到图中。
    graphs_by_id: 图的字典，包含所有图对象
    feat_names: 需要加载的特征列表
    split: 数据集分割标志
    sample_text: 样本标志
    """
    for feat in feat_names:
        # 构造CSV文件路径
        prefix = "_ABS_DATAFLOW_"
        filepath = f"/home/l1/qingao/DeepDFA/DDFA/storage/processed/bigvul/nodes_feat_{prefix}{feat}_all_limitall_10000_limitsubkeys_10000_{split}{sample_text}.csv"
        # 读取特征数据
        feat_df = pd.read_csv(filepath, index_col=0)
        
        # 将特征添加到对应的图中
        for graph_id, group in tqdm(feat_df.groupby("graph_id"), f"Adding feature {feat}"):
            if graph_id in graphs_by_id:
                g = graphs_by_id[graph_id]
                # 假设列名以 _ABS_DATAFLOW_{feat} 开头
                feat_column = next(c for c in feat_df.columns if c.startswith(f"_ABS_DATAFLOW_{feat}"))
                # 将特征转换为 tensor 并加入图的 ndata
                g.ndata[f"_ABS_DATAFLOW_{feat}"] = th.LongTensor(group[feat_column].tolist())
                
    return graphs_by_id


In [4]:
# 假设你已经有了 graphs_by_id 的图字典
graphs_by_id = load_additional_features(graphs_by_id, allfeats)

# 检查特征是否成功添加
graph_id = list(graphs_by_id.keys())[0]  # 检查第一个图
g = graphs_by_id[graph_id]
print(g.ndata.keys())  # 打印图中已有的特征


Adding feature operator: 100%|██████████| 187062/187062 [00:04<00:00, 39027.88it/s]

dict_keys(['feat', '_ABS_DATAFLOW_api', '_ABS_DATAFLOW_datatype', '_ABS_DATAFLOW_literal', '_ABS_DATAFLOW_operator'])


In [5]:
print(len(graphs_by_id))

9333


In [21]:
import sys
sys.path.append("/home/l1/qingao/DeepDFA")
import itertools
from dgl.nn.pytorch import GatedGraphConv, GlobalAttentionPooling
import torch
from torch import nn
from DDFA.code_gnn.models.base_module import BaseModule
from pytorch_lightning.utilities.cli import MODEL_REGISTRY
import logging

logger = logging.getLogger(__name__)

allfeats = [
    "api", "datatype", "literal", "operator"
]

@MODEL_REGISTRY
class FlowGNNGGNNModule(BaseModule):
    def __init__(self,
                 feat,
                 input_dim,
                 hidden_dim,
                 n_steps,
                 num_output_layers,
                 label_style="graph",
                 concat_all_absdf=False,
                 encoder_mode=False,
                 code_embedding_dim=768,  # CodeT5 embedding dimension
                 **kwargs):
        super().__init__(**kwargs)
        self.save_hyperparameters()

        if "_ABS_DATAFLOW" in feat:
            feat = "_ABS_DATAFLOW"
        self.feature_keys = {
            "feature": feat,
        }

        self.input_dim = input_dim
        self.concat_all_absdf = concat_all_absdf

        # Feature extractors
        embedding_dim = hidden_dim
        if self.concat_all_absdf:
            self.all_embeddings = nn.ModuleDict({
                of: nn.Embedding(input_dim, embedding_dim) for of in allfeats
            })
            embedding_dim *= len(allfeats)
            hidden_dim *= len(allfeats)
        else:
            self.embedding = nn.Embedding(input_dim, embedding_dim)

        # Graph stage
        self.ggnn = GatedGraphConv(in_feats=embedding_dim,
                                   out_feats=hidden_dim,
                                   n_steps=n_steps,
                                   n_etypes=1)

        # CodeT5 integration
        self.code_embedding_dim = code_embedding_dim

        # Token-level aggregation (e.g., mean-pooling)
        self.token_aggregation = nn.GRU(code_embedding_dim, 512, batch_first=True)



        


    def forward(self, graph, extrafeats):
        """
        graph: DGL graph object
        extrafeats: extra features (node-related features)
        code_embeddings: precomputed embeddings from CodeT5 for the 'code' in nodes
        """
        # 获取图中每个节点的 code embeddings
        code_embeddings = graph.ndata['feat']  # 假设这已经是 [node_num, seq_len, hid_dim] 的形状

        # 获取特征的嵌入
        if self.concat_all_absdf:
            cfeats = []
            for otherfeat in allfeats:
                feat = graph.ndata[f"_ABS_DATAFLOW_{otherfeat}"]
                cfeats.append(self.all_embeddings[otherfeat](feat))
            feat_embed = torch.cat(cfeats, dim=1)  # 特征嵌入
        else:
            feat = graph.ndata[self.feature_keys["feature"]]
            feat_embed = self.embedding(feat)

        # Graph learning stage (GGNN)
        ggnn_out = self.ggnn(graph, feat_embed)
        # print("GGNN Output Size:", ggnn_out.size())

        # Token-level aggregation (reduce sequence dimension)
        # 这里假设 code_embeddings 是 [node_num, seq_len, hid_dim]
        # 我们需要将其转换为适合 GRU 的形状
        code_embeddings = code_embeddings.view(-1, code_embeddings.size(1), self.code_embedding_dim)  # [node_num, seq_len, hid_dim]

        # 进行 GRU 聚合
        output, code_embeddings_agg = self.token_aggregation(code_embeddings)

        # Expand aggregated CodeT5 embeddings to match GGNN output size
        code_embeddings = code_embeddings_agg.squeeze(0)
        # print('code_embeddings',code_embeddings.size())

        # Concatenate GGNN output, node features, and CodeT5 embeddings
        out = torch.cat([ggnn_out, feat_embed,code_embeddings], dim=-1)
        # If you're generating node-level outputs, just return out
        logits = out  # Remove pooling if not needed

        return logits


In [22]:
import types
import torch
import transformers
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss
import numpy as np
import sys
sys.path.append('/home/l1/qingao/DeepDFA/CodeT5')
from modeling_t5 import T5ForConditionalGeneration

class FLOWT5(T5ForConditionalGeneration):
    def __init__(self, config,flow_gnn):
        super().__init__(config)
        self.flow_gnn = flow_gnn
        self.wrap_encoder()

    def forward_(self, **kwargs):
        if 'input_ids' in kwargs:
            kwargs['input_ids'] = kwargs['input_ids'].view(kwargs['input_ids'].size(0), -1)
        if 'attention_mask' in kwargs:
            kwargs['attention_mask'] = kwargs['attention_mask'].view(kwargs['attention_mask'].size(0), -1)

        return super(FLOWT5, self).forward(
            **kwargs
        )

    # We need to resize as B x (N * L) instead of (B * N) x L here
    # because the T5 forward method uses the input tensors to infer
    # dimensions used in the decoder.
    # EncoderWrapper resizes the inputs as (B * N) x L.
    def forward(self, input_ids=None, attention_mask=None, graph=None, **kwargs):
    # 获取 ggnn_output
        ggnn_output = None
        padding_length = 200
        if graph is not None:
            ggnn_output = torch.zeros(input_ids.size(0), padding_length, 768)
            num_nodes = []
            i = 0
            for g in graph:
                
                out = self.flow_gnn(g, {})  # ggnn_output 的形状为 (batch_size,seq, 512)
                if out.size(0)<padding_length:
                    ggnn_output[i, :out.size(0)] = out 
                else:
                    ggnn_output[i, :, :] = out[:padding_length,:] 
                num_nodes.append(out.size(0))
                i +=  1
        self.encoder.gnn_out = ggnn_output

        # print(ggnn_output.size())
        # 保证 input_ids 是 2D tensor，确保其类型为 LongTensor
        if input_ids is not None:
            # 如果 input_ids 是 3D 维度，需要展平为 2D
            if input_ids.dim() == 3:
                self.encoder.n_passages = input_ids.size(1)
            input_ids = input_ids.view(input_ids.size(0), -1).long()  # 确保 input_ids 是 LongTensor
            if ggnn_output is not None:
                padding = torch.zeros(input_ids.size(0), padding_length, dtype=input_ids.dtype, device=input_ids.device)  # (batch_size, num_nodes)

        # 在最后一个维度上拼接 input_ids 和 padding
    
                input_ids = torch.cat((input_ids, padding), dim=1)

        if attention_mask is not None:
            attention_mask = attention_mask.view(attention_mask.size(0), -1)  # 确保 attention_mask 也是 2D
            if ggnn_output is not None:
                        # 创建与 input_ids 相同大小的填充
                padding = torch.ones(input_ids.size(0), padding_length, dtype=input_ids.dtype, device=input_ids.device)  # (batch_size, num_nodes)
                for i, num in enumerate(num_nodes):
                    # 将前 num 个元素设置为 1
                    padding[i, :num] = 1
                # 在最后一个维度上拼接 attention_mask 和 padding
                attention_mask = torch.cat((attention_mask, padding), dim=1)

        # 将 ggnn_output 通过 kwargs 传递到 EncoderWrapper 中处理
        # print('input_ids',input_ids.size())
        # print('attention_mask',attention_mask.size())
        return super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask, # 将 ggnn_output 传入到 EncoderWrapper 进行后续处理
            **kwargs,
        )

    # We need to resize the inputs here, as the generate method expect 2D tensors
    def generate(self, input_ids, attention_mask, max_length, graph,**kwargs):
        self.encoder.n_passages = input_ids.size(1)
        ggnn_output = None
        padding_length = 200
        if graph is not None:
            ggnn_output = torch.zeros(input_ids.size(0), padding_length, 768)
            num_nodes = []
            i = 0
            for g in graph:
                
                out = self.flow_gnn(g, {})  # ggnn_output 的形状为 (batch_size,seq, 512)
                if out.size(0)<padding_length:
                    ggnn_output[i, :out.size(0)] = out 
                else:
                    ggnn_output[i, :, :] = out[:padding_length,:] 
                num_nodes.append(out.size(0))
                i += 1
        self.encoder.gnn_out = ggnn_output
        # 保证 input_ids 是 2D tensor，确保其类型为 LongTensor
        if input_ids is not None:
            # 如果 input_ids 是 3D 维度，需要展平为 2D
            if input_ids.dim() == 3:
                self.encoder.n_passages = input_ids.size(1)
            input_ids = input_ids.view(input_ids.size(0), -1).long()  # 确保 input_ids 是 LongTensor
            if ggnn_output is not None:
                padding = torch.zeros(input_ids.size(0), padding_length, dtype=input_ids.dtype, device=input_ids.device)  # (batch_size, num_nodes)

        # 在最后一个维度上拼接 input_ids 和 padding
    
                input_ids = torch.cat((input_ids, padding), dim=1)
        if attention_mask is not None:
            attention_mask = attention_mask.view(attention_mask.size(0), -1)  # 确保 attention_mask 也是 2D
            if ggnn_output is not None:
                        # 创建与 input_ids 相同大小的填充
                padding = torch.ones(input_ids.size(0), padding_length, dtype=input_ids.dtype, device=input_ids.device)  # (batch_size, num_nodes)
                for i, num in enumerate(num_nodes):
                    # 将前 num 个元素设置为 1
                    padding[i, :num] = 1
                # 在最后一个维度上拼接 attention_mask 和 padding
                attention_mask = torch.cat((attention_mask, padding), dim=1)
        
            # 将 ggnn_output 添加到 kwargs 中，以便在编码器中使用
        return super().generate(
            input_ids=input_ids.view(input_ids.size(0), -1),
            attention_mask=attention_mask.view(attention_mask.size(0), -1),
            max_length=max_length,
            **kwargs
        )

    def wrap_encoder(self, use_checkpoint=False):
        """
        Wrap T5 encoder to obtain a Fusion-in-Decoder model.
        """
        self.encoder = EncoderWrapper(self.encoder, use_checkpoint=use_checkpoint)

    def unwrap_encoder(self):
        """
        Unwrap Fusion-in-Decoder encoder, useful to load T5 weights.
        """
        self.encoder = self.encoder.encoder
        block = []
        for mod in self.encoder.block:
            block.append(mod.module)
        block = nn.ModuleList(block)
        self.encoder.block = block

    def load_t5(self, state_dict):
        self.unwrap_encoder()
        self.load_state_dict(state_dict,strict=False)
        self.wrap_encoder()

    def set_checkpoint(self, use_checkpoint):
        """
        Enable or disable checkpointing in the encoder.
        See https://pytorch.org/docs/stable/checkpoint.html
        """
        for mod in self.encoder.encoder.block:
            mod.use_checkpoint = use_checkpoint

    def reset_score_storage(self):
        """
        Reset score storage, only used when cross-attention scores are saved
        to train a retriever.
        """
        for mod in self.decoder.block:
            mod.layer[1].EncDecAttention.score_storage = None

    def get_crossattention_scores(self, context_mask):
        """
        Cross-attention scores are aggregated to obtain a single scalar per
        passage. This scalar can be seen as a similarity score between the
        question and the input passage. It is obtained by averaging the
        cross-attention scores obtained on the first decoded token over heads,
        layers, and tokens of the input passage.

        More details in Distilling Knowledge from Reader to Retriever:
        https://arxiv.org/abs/2012.04584.
        """
        scores = []
        n_passages = context_mask.size(1)
        for mod in self.decoder.block:
            scores.append(mod.layer[1].EncDecAttention.score_storage)
        scores = torch.cat(scores, dim=2)
        bsz, n_heads, n_layers, _ = scores.size()
        # batch_size, n_head, n_layers, n_passages, text_maxlength
        scores = scores.view(bsz, n_heads, n_layers, n_passages, -1)
        scores = scores.masked_fill(~context_mask[:, None, None], 0.)
        scores = scores.sum(dim=[1, 2, 4])
        ntokens = context_mask.sum(dim=[2]) * n_layers * n_heads
        scores = scores/ntokens
        return scores

    def overwrite_forward_crossattention(self):
        """
        Replace cross-attention forward function, only used to save
        cross-attention scores.
        """
        for mod in self.decoder.block:
            attn = mod.layer[1].EncDecAttention
            attn.forward = types.MethodType(cross_attention_forward, attn)

class EncoderWrapper(torch.nn.Module):
    def __init__(self, encoder, use_checkpoint=False):
        super().__init__()
        self.main_input_name = "input_ids"
        # self.linear = nn.Linear(896, 768)  # 用于调整拼接后的 hidden state 大小
        self.encoder = encoder
        self.gnn_out = None
        apply_checkpoint_wrapper(self.encoder, use_checkpoint)

        # Query 和 Key 的线性变换，用于计算注意力权重
        # self.query_layer = nn.Linear(768, 512)  # 用于对T5的hidden state进行投影
        # self.key_layer = nn.Linear(768, 512)    # 用于对GNN的输出进行投影

    def forward(self, input_ids=None, attention_mask=None, ggnn_output=None, **kwargs):
        # print(input_ids.size(),attention_mask.size())
        outputs = self.encoder(input_ids[:,:512], attention_mask[:,:512], **kwargs)
        # print('input_ids', input_ids.size())
        if self.gnn_out is not None:
            self.gnn_out = self.gnn_out.to('cuda:1')
            encoder_hidden_states = outputs.last_hidden_state  # (batch_size, seq_len, hidden_dim)
            # print(encoder_hidden_states.size())
            # ggnn_output 是 (batch_size, 768)，我们需要将其转换为 query，变为 (batch_size, 1, 768)

            encoder_hidden_states = torch.cat((encoder_hidden_states, self.gnn_out), dim=1)  # 替换填充部分
            # print(encoder_hidden_states.size())


            # print('outputs.last_hidden_state',outputs.last_hidden_state)
            outputs.last_hidden_state = encoder_hidden_states

        # print(outputs)
        return outputs
class CheckpointWrapper(torch.nn.Module):
    def __init__(self, module, use_checkpoint=False):
        super().__init__()
        self.module = module
        self.use_checkpoint = use_checkpoint

    def forward(self, hidden_states, attention_mask, position_bias, **kwargs):
        if self.use_checkpoint and self.training:
            kwargs = {k: v for k, v in kwargs.items() if v is not None}
            def custom_forward(*inputs):
                output = self.module(*inputs, **kwargs)
                empty = torch.tensor(
                    [],
                    dtype=torch.float,
                    device=output[0].device,
                    requires_grad=True)
                output = tuple(x if x is not None else empty for x in output)
                return output

            output = torch.utils.checkpoint.checkpoint(
                custom_forward,
                hidden_states,
                attention_mask,
                position_bias
            )
            output = tuple(x if x.size() != 0 else None for x in output)
        else:
            output = self.module(hidden_states, attention_mask, position_bias, **kwargs)
        return output

def apply_checkpoint_wrapper(t5stack, use_checkpoint):
    block = []
    for mod in t5stack.block:
        wrapped_mod = CheckpointWrapper(mod, use_checkpoint)
        block.append(wrapped_mod)
    block = nn.ModuleList(block)
    t5stack.block = block

def cross_attention_forward(
        self,
        input,
        mask=None,
        kv=None,
        position_bias=None,
        past_key_value_state=None,
        head_mask=None,
        query_length=None,
        use_cache=False,
        output_attentions=False,
    ):
    assert(kv != None)
    assert(head_mask == None)
    assert(position_bias != None or self.has_relative_attention_bias)

    bsz, qlen, dim = input.size()
    n_heads, d_heads = self.n_heads, self.d_kv
    klen = kv.size(1)

    q = self.q(input).view(bsz, -1, n_heads, d_heads).transpose(1, 2)
    if past_key_value_state == None:
        k = self.k(kv).view(bsz, -1, n_heads, d_heads).transpose(1, 2)
        v = self.v(kv).view(bsz, -1, n_heads, d_heads).transpose(1, 2)
    else:
        k, v = past_key_value_state

    scores = torch.einsum("bnqd,bnkd->bnqk", q, k)

    if mask is not None:
       scores += mask

    if position_bias is None:
        position_bias = self.compute_bias(qlen, klen)
    scores += position_bias

    if self.score_storage is None:
        self.score_storage = scores

    attn = F.softmax(scores.float(), dim=-1).type_as(scores)
    attn = F.dropout(attn, p=self.dropout, training=self.training)

    output = torch.matmul(attn, v)
    output = output.transpose(1, 2).contiguous().view(bsz, -1, self.inner_dim)
    output = self.o(output)

    if use_cache:
        output = (output,) + ((k, v),)
    else:
        output = (output,) + (None,)

    if output_attentions:
        output = output + (attn,)

    if self.has_relative_attention_bias:
        output = output + (position_bias,)

    return output




import torch
from transformers import AutoTokenizer
import transformers
import dgl
tokenizer = AutoTokenizer.from_pretrained(
    '/home/hdd/qingao/cache/huggingface/transformers/models--Salesforce--codet5-base/snapshots/4078456db09ba972a3532827a0b5df4da172323c'
    )
tokenizer.add_tokens(["Vul_Start","Vul_End"])
model = transformers.T5ForConditionalGeneration.from_pretrained(
    '/home/hdd/qingao/cache/huggingface/transformers/models--Salesforce--codet5-base/snapshots/4078456db09ba972a3532827a0b5df4da172323c'
    )
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load('/home/hdd/qingao/DeepDFA/CodeT5/saved_models/repair/codeT5/checkpoint-best-acc/pytorch_model.bin'))
config = model.config


input_dim = 8
feat = "_ABS_DATAFLOW_datatype_all_limitall_1000_limitsubkeys_1000"
gtype = "cfg"
label_style = "graph"
dsname = "bigvul"
node_type_feat = None
concat_all_absdf = True
hidden_dim = 32
n_steps = 5
num_output_layers = 3

flowgnn_model = FlowGNNGGNNModule(
    feat,
    input_dim,
    hidden_dim,
    n_steps,
    num_output_layers,
    label_style=label_style,
    # freeze_graph=False,
    # append_dataflow="before_graph",
    # codebert_feat=None,
    # doc2vec_feat=None,
    # glove_feat=None,
    # num_node_types=flowgnn_datamodule.num_node_types,
    # node_type_feat=node_type_feat,
    # just_codebert=False,
    concat_all_absdf=concat_all_absdf,
    # undersample_node_on_loss_factor=None,
    # test_every=False,
    # tune_nni=False,
    # positive_weight=None,
    encoder_mode=True,
)


flow_model = FLOWT5(config,flow_gnn=flowgnn_model)
flow_model.load_t5(model.state_dict())

/tmp/ipykernel_1015421/1697295338.py:351: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/home/hdd/qingao/DeepDFA/CodeT5/saved_models/repair

In [11]:
flow_model

FLOWT5(
  (shared): Embedding(32102, 768)
  (encoder): EncoderWrapper(
    (encoder): T5Stack(
      (embed_tokens): Embedding(32102, 768)
      (block): ModuleList(
        (0): CheckpointWrapper(
          (module): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(in_features=768, out_features=768, bias=False)
                  (k): Linear(in_features=768, out_features=768, bias=False)
                  (v): Linear(in_features=768, out_features=768, bias=False)
                  (o): Linear(in_features=768, out_features=768, bias=False)
                  (relative_attention_bias): Embedding(32, 12)
                )
                (layer_norm): T5LayerNorm()
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (1): T5LayerFF(
                (DenseReluDense): T5DenseActDense(
                  (wi): Linear(in_features=768, out_features=3072,

In [12]:
import json
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup

# 读取训练数据
train_data = []
with open('/home/l1/qingao/DeepDFA/DDFA/storage/external/train_data.json', 'r') as f:
    for line in f:
        train_data.append(json.loads(line))

# 获取有效的图ID
valid_index = graph_labels["graph_ids"]

val_data = []
with open('/home/l1/qingao/DeepDFA/DDFA/storage/external/val_data.json', 'r') as f:
    for line in f:
        val_data.append(json.loads(line))

# 获取有效的图ID
valid_index = graph_labels["graph_ids"]


# 筛选出有效的数据
filtered_data = [item for item in train_data if item.get('Unnamed: 0') in valid_index]
filtered_val_data = [item for item in val_data if item.get('Unnamed: 0') in valid_index]


train_df = pd.DataFrame(filtered_data)  # 将 filtered_data 转换为 DataFrame
val_df = pd.DataFrame(filtered_val_data)  # 将 filtered_val_data 转换为 DataFrame

# 将 Pandas DataFrame 转换为 Hugging Face 的 Dataset 类型
train_Ds = Dataset.from_pandas(train_df)
val_Ds = Dataset.from_pandas(val_df)

def process_func(example):
    MAX_LENGTH = 512
    source = ''.join(example['source']) if isinstance(example['source'], list) else example['source']
    target = ''.join(example['target']) if isinstance(example['target'], list) else example['target']
    inputs = tokenizer(source, truncation=True, max_length=MAX_LENGTH, padding='max_length')
    labels = tokenizer(target, truncation=True, max_length=MAX_LENGTH, padding='max_length')
    index = example['Unnamed: 0']
    return {
        "input_ids": inputs['input_ids'],
        "attention_mask": inputs['attention_mask'],
        "labels": labels['input_ids'],
        "index" : torch.tensor(index)
    }

train_ds = train_Ds.map(process_func, batched=False, remove_columns=train_Ds.column_names)
val_ds = val_Ds.map(process_func, batched=False, remove_columns=val_Ds.column_names)
# 输出筛选后的数据
train_dataloader = DataLoader(train_ds, collate_fn=default_data_collator, batch_size=16, pin_memory=True)
eval_dataloader = DataLoader(val_ds, collate_fn=default_data_collator, batch_size=1, pin_memory=True)


Map: 100%|██████████| 903/903 [00:02<00:00, 308.84 examples/s]


In [13]:
len(filtered_data)

6548

In [14]:
train_ds = train_Ds.map(process_func, batched=False, remove_columns=train_Ds.column_names)
val_ds = val_Ds.map(process_func, batched=False, remove_columns=val_Ds.column_names)

Map:   1%|          | 49/6548 [00:00<00:14, 455.06 examples/s]

Map: 100%|██████████| 903/903 [00:02<00:00, 328.65 examples/s]


In [15]:
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
train_dataloader = DataLoader(train_ds, collate_fn=default_data_collator, batch_size=16, pin_memory=True)
eval_dataloader = DataLoader(val_ds, collate_fn=default_data_collator, batch_size=1, pin_memory=True)

In [16]:
import re
def clean_tokens(tokens):
    tokens = tokens.replace("<pad>", "")
    tokens = tokens.replace("<s>", "")
    tokens = tokens.replace("</s>", "")
    tokens = tokens.replace(' ','')
    tokens = tokens.replace("</s>", "")
    tokens = tokens.replace("<start>", "").replace('<end>','')
    tokens = re.sub(r'\s+', ' ', tokens)
    tokens = tokens.strip()
    return tokens
def eval(model,eval_dataloader,graphs_by_id):

    bar = tqdm(eval_dataloader, total=len(eval_dataloader), desc="eval")
    model.to('cuda:1')
    model.eval()
    exmatch = 0
    for step, batch in enumerate(bar):
        input_ids = batch['input_ids'].to('cuda:1')
        attention_mask = batch['attention_mask'].to('cuda:1')
        labels = batch['labels'].to('cuda:1')
        index = batch['index'].to('cuda:1')

        index_list = index.tolist()

        if graphs_by_id is None:
            graphs = None
        else:
            graphs = [graphs_by_id[i].to('cuda:1') for i in index_list if i in graphs_by_id]

        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask,
                                max_length = 128,
                                graph=graphs if graphs else None)
        for output,label in zip(outputs,labels):
            if(clean_tokens(tokenizer.decode(output,skip_special_tokens=True))==clean_tokens(tokenizer.decode(label,skip_special_tokens=True))):
                exmatch+=1
                # print('FlowT5:',tokenizer.decode(output,skip_special_tokens=True))
                # print('T5',tokenizer.decode(output_code,skip_special_tokens=True))
                # print('answer',tokenizer.decode(label,skip_special_tokens=True))
    return exmatch

In [23]:
from transformers import AdamW, get_linear_schedule_with_warmup
import os 
# # 冻结 CodeT5 的参数
# for param in flow_model.shared.parameters():
#     param.requires_grad = False

# for param in flow_model.encoder.parameters():
#     param.requires_grad = False

# # 如果 GNN 部分是单独的模块，确保它的参数不被冻结
# # 这里假设 GNN 部分的名称为 `gnn`，请根据实际情况修改
# for param in flow_model.flow_gnn.parameters():
#     param.requires_grad = False  # 确保 GNN 的参数可训练


# for param in flow_model.decoder.parameters():  # 确保流入解码器的参数被冻结
#     param.requires_grad = True

# # 定义学习率和优化器

criterion = nn.CrossEntropyLoss()
lr = 1e-5
num_epochs = 5
best_em = 0
# 冻结 CodeT5 encoder 和 decoder 的参数
output_dir = 'home/hdd/qingao/RMGArepair/saved_model/step5_nodes200_dec_enc_dec_gnnf_wd1e-4_2/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for param in flow_model.flow_gnn.parameters():
    param.requires_grad = True

for param in flow_model.encoder.parameters():
    param.requires_grad = False

for param in flow_model.decoder.parameters():
    param.requires_grad = True

# 定义不进行权重衰减的参数
no_decay = ['bias', 'LayerNorm.weight']

# 为优化器分组参数
optimizer_grouped_parameters = [
    {'params': [p for n, p in flow_model.named_parameters() if not any(nd in n for nd in no_decay) and p.requires_grad],
     'weight_decay': 1e-4
     },
    {'params': [p for n, p in flow_model.named_parameters() if any(nd in n for nd in no_decay) and p.requires_grad], 
     'weight_decay': 0.0}
]

# 定义 AdamW 优化器
optimizer = AdamW(optimizer_grouped_parameters, lr=lr, eps=1e-8)
# for name, param in flow_model.flow_gnn.named_parameters():
#     if param.requires_grad:
#         print(f"Parameter: {name}, Value: {param.data}, Gradient: {param.grad}")
from transformers import get_linear_schedule_with_warmup

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=400,
    num_training_steps=(len(train_dataloader) * num_epochs),
)
# scheduler = get_linear_schedule_with_warmup(optimizer,
#                                             num_warmup_steps=args.warmup_steps,
#                                             num_training_steps=num_train_optimization_steps)


gradient_accumulation_steps = 1
# 训练循环
for cur_epoch in range(0, num_epochs):
    bar = tqdm(train_dataloader, total=len(train_dataloader), desc="Training")
    nb_tr_examples, nb_tr_steps, tr_loss = 0, 0, 0
    flow_model.to('cuda:1')
    flow_model.train()
    
    for step, batch in enumerate(bar):
        input_ids = batch['input_ids'].to('cuda:1')
        attention_mask = batch['attention_mask'].to('cuda:1')
        labels = batch['labels'].to('cuda:1')
        index = batch['index'].to('cuda:1')

        index_list = index.tolist()

        if graphs_by_id is None:
            graphs = None
        else:
            graphs = [graphs_by_id[i].to('cuda:1') for i in index_list if i in graphs_by_id]

            outputs = flow_model(input_ids=input_ids, attention_mask=attention_mask,
                                 labels=labels,
                                 graph=graphs if graphs else None)

        loss = outputs.loss
        tr_loss += loss.item()

        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        loss.backward()

        if nb_tr_steps % gradient_accumulation_steps == 0:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            # print("Flow GNN Parameters and Gradients:")
            # for name, param in flow_model.flow_gnn.named_parameters():
            #     if param.requires_grad:
            #         print(f"Parameter: {name}, Value: {param.data}, Gradient: {param.grad}")
            # break
            # 计算和打印平均损失
            train_loss = round(tr_loss / nb_tr_steps, 4) if nb_tr_steps > 0 else 0
            bar.set_description("[{}] Train loss {}".format(cur_epoch, round(train_loss, 3)))
    # em = eval(flow_model,eval_dataloader,graphs_by_id)
    # print(em)
    # if em>best_em:
    #     best_em = em
    #     output_model_file = os.path.join(output_dir, "pytorch_model.bin")
    #     torch.save(flow_model.state_dict(), output_model_file)
    #     print("Save the best acc model into %s", output_model_file)
for param in flow_model.flow_gnn.parameters():
    param.requires_grad = False

for param in flow_model.encoder.parameters():
    param.requires_grad = False

for param in flow_model.decoder.parameters():
    param.requires_grad = True

# 定义不进行权重衰减的参数
no_decay = ['bias', 'LayerNorm.weight']

# 为优化器分组参数
optimizer_grouped_parameters = [
    {'params': [p for n, p in flow_model.named_parameters() if not any(nd in n for nd in no_decay) and p.requires_grad],
     'weight_decay': 1e-2},
    {'params': [p for n, p in flow_model.named_parameters() if any(nd in n for nd in no_decay) and p.requires_grad], 
     'weight_decay': 0.0}
]
lr = 1e-4
# 定义 AdamW 优化器
optimizer = AdamW(optimizer_grouped_parameters, lr=lr, eps=1e-8)
# for name, param in flow_model.flow_gnn.named_parameters():
#     if param.requires_grad:
#         print(f"Parameter: {name}, Value: {param.data}, Gradient: {param.grad}")
from transformers import get_linear_schedule_with_warmup

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=400,
    num_training_steps=(len(train_dataloader) * num_epochs),
)
# scheduler = get_linear_schedule_with_warmup(optimizer,
#                                             num_warmup_steps=args.warmup_steps,
#                                             num_training_steps=num_train_optimization_steps)


gradient_accumulation_steps = 1
# 训练循环
for cur_epoch in range(0, 10):
    bar = tqdm(train_dataloader, total=len(train_dataloader), desc="Training")
    nb_tr_examples, nb_tr_steps, tr_loss = 0, 0, 0
    flow_model.to('cuda:1')
    flow_model.train()
    
    for step, batch in enumerate(bar):
        input_ids = batch['input_ids'].to('cuda:1')
        attention_mask = batch['attention_mask'].to('cuda:1')
        labels = batch['labels'].to('cuda:1')
        index = batch['index'].to('cuda:1')

        index_list = index.tolist()

        if graphs_by_id is None:
            graphs = None
        else:
            graphs = [graphs_by_id[i].to('cuda:1') for i in index_list if i in graphs_by_id]

            outputs = flow_model(input_ids=input_ids, attention_mask=attention_mask,
                                 labels=labels,
                                 graph=graphs if graphs else None)
            


        loss = outputs.loss
        tr_loss += loss.item()

        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        loss.backward()

        if nb_tr_steps % gradient_accumulation_steps == 0:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            # print("Flow GNN Parameters and Gradients:")
            # for name, param in flow_model.flow_gnn.named_parameters():
            #     if param.requires_grad:
            #         print(f"Parameter: {name}, Value: {param.data}, Gradient: {param.grad}")
            # break
            # 计算和打印平均损失
            train_loss = round(tr_loss / nb_tr_steps, 4) if nb_tr_steps > 0 else 0
            bar.set_description("[{}] Train loss {}".format(cur_epoch, round(train_loss, 3)))
    em = eval(flow_model,eval_dataloader,graphs_by_id)
    print(em)
    if em>best_em:
        best_em = em
        output_model_file = os.path.join(output_dir, "pytorch_model.bin")
        torch.save(flow_model.state_dict(), output_model_file)
        print("Save the best acc model into %s", output_model_file)



Training:   0%|          | 0/410 [00:00<?, ?it/s]

eval: 100%|██████████| 903/903 [15:00<00:00,  1.00it/s]


150
Save the best acc model into %s home/hdd/qingao/RMGArepair/saved_model/step5_nodes200_dec_enc_dec_gnnf_wd1e-4_2/pytorch_model.bin


eval: 100%|██████████| 903/903 [15:35<00:00,  1.04s/it]


147


...

183


eval: 100%|██████████| 903/903 [14:02<00:00,  1.07it/s]


183


eval: 100%|██████████| 903/903 [13:47<00:00,  1.09it/s]

183


In [15]:
import types
import torch
import transformers
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss
import numpy as np
import sys
sys.path.append('/home/l1/qingao/DeepDFA/CodeT5')
from modeling_t5 import T5ForConditionalGeneration

class FLOWT5(T5ForConditionalGeneration):
    def __init__(self, config,flow_gnn):
        super().__init__(config)
        self.flow_gnn = flow_gnn
        self.wrap_encoder()

    def forward_(self, **kwargs):
        if 'input_ids' in kwargs:
            kwargs['input_ids'] = kwargs['input_ids'].view(kwargs['input_ids'].size(0), -1)
        if 'attention_mask' in kwargs:
            kwargs['attention_mask'] = kwargs['attention_mask'].view(kwargs['attention_mask'].size(0), -1)

        return super(FLOWT5, self).forward(
            **kwargs
        )

    # We need to resize as B x (N * L) instead of (B * N) x L here
    # because the T5 forward method uses the input tensors to infer
    # dimensions used in the decoder.
    # EncoderWrapper resizes the inputs as (B * N) x L.
    def forward(self, input_ids=None, attention_mask=None, graph=None, **kwargs):
    # 获取 ggnn_output
        ggnn_output = None
        padding_length = 200
        if graph is not None:
            ggnn_output = torch.zeros(input_ids.size(0), padding_length, 768)
            num_nodes = []
            i = 0
            for g in graph:
                
                out = self.flow_gnn(g, {})  # ggnn_output 的形状为 (batch_size,seq, 512)
                if out.size(0)<padding_length:
                    ggnn_output[i, :out.size(0)] = out 
                else:
                    ggnn_output[i, :, :] = out[:padding_length,:] 
                num_nodes.append(out.size(0))
                i += 1
        self.encoder.gnn_out = ggnn_output

        # print(ggnn_output.size())
        # 保证 input_ids 是 2D tensor，确保其类型为 LongTensor
        if input_ids is not None:
            # 如果 input_ids 是 3D 维度，需要展平为 2D
            if input_ids.dim() == 3:
                self.encoder.n_passages = input_ids.size(1)
            input_ids = input_ids.view(input_ids.size(0), -1).long()  # 确保 input_ids 是 LongTensor
            if ggnn_output is not None:
                padding = torch.zeros(input_ids.size(0), padding_length, dtype=input_ids.dtype, device=input_ids.device)  # (batch_size, num_nodes)

        # 在最后一个维度上拼接 input_ids 和 padding
    
                input_ids = torch.cat((input_ids, padding), dim=1)

        if attention_mask is not None:
            attention_mask = attention_mask.view(attention_mask.size(0), -1)  # 确保 attention_mask 也是 2D
            if ggnn_output is not None:
                        # 创建与 input_ids 相同大小的填充
                padding = torch.ones(input_ids.size(0), padding_length, dtype=input_ids.dtype, device=input_ids.device)  # (batch_size, num_nodes)
                for i, num in enumerate(num_nodes):
                    # 将前 num 个元素设置为 1
                    padding[i, :num] = 1
                # 在最后一个维度上拼接 attention_mask 和 padding
                attention_mask = torch.cat((attention_mask, padding), dim=1)

        # 将 ggnn_output 通过 kwargs 传递到 EncoderWrapper 中处理
        # print('input_ids',input_ids.size())
        # print('attention_mask',attention_mask.size())
        return super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask, # 将 ggnn_output 传入到 EncoderWrapper 进行后续处理
            **kwargs,
        )

    # We need to resize the inputs here, as the generate method expect 2D tensors
    def generate(self, input_ids, attention_mask, max_length, graph,**kwargs):
        self.encoder.n_passages = input_ids.size(1)
        ggnn_output = None
        padding_length = 200
        if graph is not None:
            ggnn_output = torch.zeros(input_ids.size(0), padding_length, 768)
            num_nodes = []
            i = 0
            for g in graph:
                
                out = self.flow_gnn(g, {})  # ggnn_output 的形状为 (batch_size,seq, 512)
                if out.size(0)<padding_length:
                    ggnn_output[i, :out.size(0)] = out 
                else:
                    ggnn_output[i, :, :] = out[:padding_length,:] 
                num_nodes.append(out.size(0))
                i += 1
        self.encoder.gnn_out = ggnn_output
        # 保证 input_ids 是 2D tensor，确保其类型为 LongTensor
        if input_ids is not None:
            # 如果 input_ids 是 3D 维度，需要展平为 2D
            if input_ids.dim() == 3:
                self.encoder.n_passages = input_ids.size(1)
            input_ids = input_ids.view(input_ids.size(0), -1).long()  # 确保 input_ids 是 LongTensor
            if ggnn_output is not None:
                padding = torch.zeros(input_ids.size(0), padding_length, dtype=input_ids.dtype, device=input_ids.device)  # (batch_size, num_nodes)

        # 在最后一个维度上拼接 input_ids 和 padding
    
                input_ids = torch.cat((input_ids, padding), dim=1)
        if attention_mask is not None:
            attention_mask = attention_mask.view(attention_mask.size(0), -1)  # 确保 attention_mask 也是 2D
            if ggnn_output is not None:
                        # 创建与 input_ids 相同大小的填充
                padding = torch.ones(input_ids.size(0), padding_length, dtype=input_ids.dtype, device=input_ids.device)  # (batch_size, num_nodes)
                for i, num in enumerate(num_nodes):
                    # 将前 num 个元素设置为 1
                    padding[i, :num] = 1
                # 在最后一个维度上拼接 attention_mask 和 padding
                attention_mask = torch.cat((attention_mask, padding), dim=1)
        
            # 将 ggnn_output 添加到 kwargs 中，以便在编码器中使用
        return super().generate(
            input_ids=input_ids.view(input_ids.size(0), -1),
            attention_mask=attention_mask.view(attention_mask.size(0), -1),
            max_length=max_length,
            **kwargs
        )

    def wrap_encoder(self, use_checkpoint=False):
        """
        Wrap T5 encoder to obtain a Fusion-in-Decoder model.
        """
        self.encoder = EncoderWrapper(self.encoder, use_checkpoint=use_checkpoint)

    def unwrap_encoder(self):
        """
        Unwrap Fusion-in-Decoder encoder, useful to load T5 weights.
        """
        self.encoder = self.encoder.encoder
        block = []
        for mod in self.encoder.block:
            block.append(mod.module)
        block = nn.ModuleList(block)
        self.encoder.block = block

    def load_t5(self, state_dict):
        self.unwrap_encoder()
        self.load_state_dict(state_dict,strict=False)
        self.wrap_encoder()

    def set_checkpoint(self, use_checkpoint):
        """
        Enable or disable checkpointing in the encoder.
        See https://pytorch.org/docs/stable/checkpoint.html
        """
        for mod in self.encoder.encoder.block:
            mod.use_checkpoint = use_checkpoint

    def reset_score_storage(self):
        """
        Reset score storage, only used when cross-attention scores are saved
        to train a retriever.
        """
        for mod in self.decoder.block:
            mod.layer[1].EncDecAttention.score_storage = None

    def get_crossattention_scores(self, context_mask):
        """
        Cross-attention scores are aggregated to obtain a single scalar per
        passage. This scalar can be seen as a similarity score between the
        question and the input passage. It is obtained by averaging the
        cross-attention scores obtained on the first decoded token over heads,
        layers, and tokens of the input passage.

        More details in Distilling Knowledge from Reader to Retriever:
        https://arxiv.org/abs/2012.04584.
        """
        scores = []
        n_passages = context_mask.size(1)
        for mod in self.decoder.block:
            scores.append(mod.layer[1].EncDecAttention.score_storage)
        scores = torch.cat(scores, dim=2)
        bsz, n_heads, n_layers, _ = scores.size()
        # batch_size, n_head, n_layers, n_passages, text_maxlength
        scores = scores.view(bsz, n_heads, n_layers, n_passages, -1)
        scores = scores.masked_fill(~context_mask[:, None, None], 0.)
        scores = scores.sum(dim=[1, 2, 4])
        ntokens = context_mask.sum(dim=[2]) * n_layers * n_heads
        scores = scores/ntokens
        return scores

    def overwrite_forward_crossattention(self):
        """
        Replace cross-attention forward function, only used to save
        cross-attention scores.
        """
        for mod in self.decoder.block:
            attn = mod.layer[1].EncDecAttention
            attn.forward = types.MethodType(cross_attention_forward, attn)

class EncoderWrapper(torch.nn.Module):
    def __init__(self, encoder, use_checkpoint=False):
        super().__init__()
        self.main_input_name = "input_ids"
        # self.linear = nn.Linear(896, 768)  # 用于调整拼接后的 hidden state 大小
        self.encoder = encoder
        self.gnn_out = None
        apply_checkpoint_wrapper(self.encoder, use_checkpoint)

        # Query 和 Key 的线性变换，用于计算注意力权重
        # self.query_layer = nn.Linear(768, 512)  # 用于对T5的hidden state进行投影
        # self.key_layer = nn.Linear(768, 512)    # 用于对GNN的输出进行投影

    def forward(self, input_ids=None, attention_mask=None, ggnn_output=None, **kwargs):
        # print(input_ids.size(),attention_mask.size())
        outputs = self.encoder(input_ids[:,:512], attention_mask[:,:512], **kwargs)
        # print('input_ids', input_ids.size())
        if self.gnn_out is not None:
            self.gnn_out = self.gnn_out.to('cuda:1')
            encoder_hidden_states = outputs.last_hidden_state  # (batch_size, seq_len, hidden_dim)
            # print(encoder_hidden_states.size())
            # ggnn_output 是 (batch_size, 768)，我们需要将其转换为 query，变为 (batch_size, 1, 768)

            encoder_hidden_states = torch.cat((encoder_hidden_states, self.gnn_out), dim=1)  # 替换填充部分
            # print(encoder_hidden_states.size())


            # print('outputs.last_hidden_state',outputs.last_hidden_state)
            outputs.last_hidden_state = encoder_hidden_states

        # print(outputs)
        return outputs
class CheckpointWrapper(torch.nn.Module):
    def __init__(self, module, use_checkpoint=False):
        super().__init__()
        self.module = module
        self.use_checkpoint = use_checkpoint

    def forward(self, hidden_states, attention_mask, position_bias, **kwargs):
        if self.use_checkpoint and self.training:
            kwargs = {k: v for k, v in kwargs.items() if v is not None}
            def custom_forward(*inputs):
                output = self.module(*inputs, **kwargs)
                empty = torch.tensor(
                    [],
                    dtype=torch.float,
                    device=output[0].device,
                    requires_grad=True)
                output = tuple(x if x is not None else empty for x in output)
                return output

            output = torch.utils.checkpoint.checkpoint(
                custom_forward,
                hidden_states,
                attention_mask,
                position_bias
            )
            output = tuple(x if x.size() != 0 else None for x in output)
        else:
            output = self.module(hidden_states, attention_mask, position_bias, **kwargs)
        return output

def apply_checkpoint_wrapper(t5stack, use_checkpoint):
    block = []
    for mod in t5stack.block:
        wrapped_mod = CheckpointWrapper(mod, use_checkpoint)
        block.append(wrapped_mod)
    block = nn.ModuleList(block)
    t5stack.block = block

def cross_attention_forward(
        self,
        input,
        mask=None,
        kv=None,
        position_bias=None,
        past_key_value_state=None,
        head_mask=None,
        query_length=None,
        use_cache=False,
        output_attentions=False,
    ):
    assert(kv != None)
    assert(head_mask == None)
    assert(position_bias != None or self.has_relative_attention_bias)

    bsz, qlen, dim = input.size()
    n_heads, d_heads = self.n_heads, self.d_kv
    klen = kv.size(1)

    q = self.q(input).view(bsz, -1, n_heads, d_heads).transpose(1, 2)
    if past_key_value_state == None:
        k = self.k(kv).view(bsz, -1, n_heads, d_heads).transpose(1, 2)
        v = self.v(kv).view(bsz, -1, n_heads, d_heads).transpose(1, 2)
    else:
        k, v = past_key_value_state

    scores = torch.einsum("bnqd,bnkd->bnqk", q, k)

    if mask is not None:
       scores += mask

    if position_bias is None:
        position_bias = self.compute_bias(qlen, klen)
    scores += position_bias

    if self.score_storage is None:
        self.score_storage = scores

    attn = F.softmax(scores.float(), dim=-1).type_as(scores)
    attn = F.dropout(attn, p=self.dropout, training=self.training)

    output = torch.matmul(attn, v)
    output = output.transpose(1, 2).contiguous().view(bsz, -1, self.inner_dim)
    output = self.o(output)

    if use_cache:
        output = (output,) + ((k, v),)
    else:
        output = (output,) + (None,)

    if output_attentions:
        output = output + (attn,)

    if self.has_relative_attention_bias:
        output = output + (position_bias,)

    return output




import torch
from transformers import AutoTokenizer
import transformers
import dgl
tokenizer = AutoTokenizer.from_pretrained(
    '/home/hdd/qingao/cache/huggingface/transformers/models--Salesforce--codet5-base/snapshots/4078456db09ba972a3532827a0b5df4da172323c'
    )
tokenizer.add_tokens(["Vul_Start","Vul_End"])
model = transformers.T5ForConditionalGeneration.from_pretrained(
    '/home/hdd/qingao/cache/huggingface/transformers/models--Salesforce--codet5-base/snapshots/4078456db09ba972a3532827a0b5df4da172323c'
    )
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load('/home/hdd/qingao/DeepDFA/CodeT5/saved_models/repair/codeT5/checkpoint-best-acc/pytorch_model.bin'))
config = model.config


input_dim = 8
feat = "_ABS_DATAFLOW_datatype_all_limitall_1000_limitsubkeys_1000"
gtype = "cfg"
label_style = "graph"
dsname = "bigvul"
node_type_feat = None
concat_all_absdf = True
hidden_dim = 64
n_steps = 5
num_output_layers = 3

flowgnn_model = FlowGNNGGNNModule(
    feat,
    input_dim,
    hidden_dim,
    n_steps,
    num_output_layers,
    label_style=label_style,
    # freeze_graph=False,
    # append_dataflow="before_graph",
    # codebert_feat=None,
    # doc2vec_feat=None,
    # glove_feat=None,
    # num_node_types=flowgnn_datamodule.num_node_types,
    # node_type_feat=node_type_feat,
    # just_codebert=False,
    concat_all_absdf=concat_all_absdf,
    # undersample_node_on_loss_factor=None,
    # test_every=False,
    # tune_nni=False,
    # positive_weight=None,
    encoder_mode=True,
)


flow_model = FLOWT5(config,flow_gnn=flowgnn_model)
flow_model.load_t5(model.state_dict())

/tmp/ipykernel_782618/3718301340.py:351: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/home/hdd/qingao/DeepDFA/CodeT5/saved_models/repair/

In [16]:
from transformers import AdamW, get_linear_schedule_with_warmup
import os 
# # 冻结 CodeT5 的参数
# for param in flow_model.shared.parameters():
#     param.requires_grad = False

# for param in flow_model.encoder.parameters():
#     param.requires_grad = False

# # 如果 GNN 部分是单独的模块，确保它的参数不被冻结
# # 这里假设 GNN 部分的名称为 `gnn`，请根据实际情况修改
# for param in flow_model.flow_gnn.parameters():
#     param.requires_grad = False  # 确保 GNN 的参数可训练


# for param in flow_model.decoder.parameters():  # 确保流入解码器的参数被冻结
#     param.requires_grad = True

# # 定义学习率和优化器

criterion = nn.CrossEntropyLoss()
lr = 1e-4
num_epochs = 5
best_em = 0
# 冻结 CodeT5 encoder 和 decoder 的参数
output_dir = 'home/hdd/qingao/RMGArepair/saved_model/step5_200nodes_encoder_decoder_freeze_gnn/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for param in flow_model.encoder.parameters():
    param.requires_grad = False

for param in flow_model.decoder.parameters():
    param.requires_grad = True

# 定义不进行权重衰减的参数
no_decay = ['bias', 'LayerNorm.weight']

# 为优化器分组参数
optimizer_grouped_parameters = [
    {'params': [p for n, p in flow_model.named_parameters() if not any(nd in n for nd in no_decay) and p.requires_grad],
     'weight_decay': 1e-4
     },
    {'params': [p for n, p in flow_model.named_parameters() if any(nd in n for nd in no_decay) and p.requires_grad], 
     'weight_decay': 0.0}
]

# 定义 AdamW 优化器
optimizer = AdamW(optimizer_grouped_parameters, lr=lr, eps=1e-8)
# for name, param in flow_model.flow_gnn.named_parameters():
#     if param.requires_grad:
#         print(f"Parameter: {name}, Value: {param.data}, Gradient: {param.grad}")
from transformers import get_linear_schedule_with_warmup

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=400,
    num_training_steps=(len(train_dataloader) * num_epochs),
)
# scheduler = get_linear_schedule_with_warmup(optimizer,
#                                             num_warmup_steps=args.warmup_steps,
#                                             num_training_steps=num_train_optimization_steps)


gradient_accumulation_steps = 1
# 训练循环
for cur_epoch in range(0, num_epochs):
    bar = tqdm(train_dataloader, total=len(train_dataloader), desc="Training")
    nb_tr_examples, nb_tr_steps, tr_loss = 0, 0, 0
    flow_model.to('cuda:1')
    flow_model.train()
    
    for step, batch in enumerate(bar):
        input_ids = batch['input_ids'].to('cuda:1')
        attention_mask = batch['attention_mask'].to('cuda:1')
        labels = batch['labels'].to('cuda:1')
        index = batch['index'].to('cuda:1')

        index_list = index.tolist()

        if graphs_by_id is None:
            graphs = None
        else:
            graphs = [graphs_by_id[i].to('cuda:1') for i in index_list if i in graphs_by_id]

            outputs = flow_model(input_ids=input_ids, attention_mask=attention_mask,
                                 labels=labels,
                                 graph=graphs if graphs else None)

        loss = outputs.loss
        tr_loss += loss.item()

        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        loss.backward()

        if nb_tr_steps % gradient_accumulation_steps == 0:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            # print("Flow GNN Parameters and Gradients:")
            # for name, param in flow_model.flow_gnn.named_parameters():
            #     if param.requires_grad:
            #         print(f"Parameter: {name}, Value: {param.data}, Gradient: {param.grad}")
            # break
            # 计算和打印平均损失
            train_loss = round(tr_loss / nb_tr_steps, 4) if nb_tr_steps > 0 else 0
            bar.set_description("[{}] Train loss {}".format(cur_epoch, round(train_loss, 3)))
    em = eval(flow_model,eval_dataloader,graphs_by_id)
    print(em)
    if em>best_em:
        best_em = em
        output_model_file = os.path.join(output_dir, "pytorch_model.bin")
        torch.save(flow_model.state_dict(), output_model_file)
        print("Save the best acc model into %s", output_model_file)

for param in flow_model.flow_gnn.parameters():
    param.requires_grad = False


for param in flow_model.encoder.parameters():
    param.requires_grad = False

for param in flow_model.decoder.parameters():
    param.requires_grad = True

# 定义不进行权重衰减的参数
no_decay = ['bias', 'LayerNorm.weight']

# 为优化器分组参数
optimizer_grouped_parameters = [
    {'params': [p for n, p in flow_model.named_parameters() if not any(nd in n for nd in no_decay) and p.requires_grad],
     'weight_decay': 1e-2},
    {'params': [p for n, p in flow_model.named_parameters() if any(nd in n for nd in no_decay) and p.requires_grad], 
     'weight_decay': 0.0}
]
lr = 1e-4
# 定义 AdamW 优化器
optimizer = AdamW(optimizer_grouped_parameters, lr=lr, eps=1e-8)
# for name, param in flow_model.flow_gnn.named_parameters():
#     if param.requires_grad:
#         print(f"Parameter: {name}, Value: {param.data}, Gradient: {param.grad}")
from transformers import get_linear_schedule_with_warmup

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=400,
    num_training_steps=(len(train_dataloader) * num_epochs),
)
# scheduler = get_linear_schedule_with_warmup(optimizer,
#                                             num_warmup_steps=args.warmup_steps,
#                                             num_training_steps=num_train_optimization_steps)


gradient_accumulation_steps = 1
# 训练循环
for cur_epoch in range(0, 10):
    bar = tqdm(train_dataloader, total=len(train_dataloader), desc="Training")
    nb_tr_examples, nb_tr_steps, tr_loss = 0, 0, 0
    flow_model.to('cuda:1')
    flow_model.train()
    
    for step, batch in enumerate(bar):
        input_ids = batch['input_ids'].to('cuda:1')
        attention_mask = batch['attention_mask'].to('cuda:1')
        labels = batch['labels'].to('cuda:1')
        index = batch['index'].to('cuda:1')

        index_list = index.tolist()

        if graphs_by_id is None:
            graphs = None
        else:
            graphs = [graphs_by_id[i].to('cuda:1') for i in index_list if i in graphs_by_id]

            outputs = flow_model(input_ids=input_ids, attention_mask=attention_mask,
                                 labels=labels,
                                 graph=graphs if graphs else None)
            


        loss = outputs.loss
        tr_loss += loss.item()

        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        loss.backward()

        if nb_tr_steps % gradient_accumulation_steps == 0:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            # print("Flow GNN Parameters and Gradients:")
            # for name, param in flow_model.flow_gnn.named_parameters():
            #     if param.requires_grad:
            #         print(f"Parameter: {name}, Value: {param.data}, Gradient: {param.grad}")
            # break
            # 计算和打印平均损失
            train_loss = round(tr_loss / nb_tr_steps, 4) if nb_tr_steps > 0 else 0
            bar.set_description("[{}] Train loss {}".format(cur_epoch, round(train_loss, 3)))
    em = eval(flow_model,eval_dataloader,graphs_by_id)
    print(em)
    if em>best_em:
        best_em = em
        output_model_file = os.path.join(output_dir, "pytorch_model.bin")
        torch.save(flow_model.state_dict(), output_model_file)
        print("Save the best acc model into %s", output_model_file)



Training:   0%|          | 0/410 [00:00<?, ?it/s]

[0] Train loss 2.684:  13%|█▎        | 54/410 [00:47<05:11,  1.14it/s]


KeyboardInterrupt: 

In [17]:
import types
import torch
import transformers
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss
import numpy as np
import sys
sys.path.append('/home/l1/qingao/DeepDFA/CodeT5')
from modeling_t5 import T5ForConditionalGeneration

class FLOWT5(T5ForConditionalGeneration):
    def __init__(self, config,flow_gnn):
        super().__init__(config)
        self.flow_gnn = flow_gnn
        self.wrap_encoder()

    def forward_(self, **kwargs):
        if 'input_ids' in kwargs:
            kwargs['input_ids'] = kwargs['input_ids'].view(kwargs['input_ids'].size(0), -1)
        if 'attention_mask' in kwargs:
            kwargs['attention_mask'] = kwargs['attention_mask'].view(kwargs['attention_mask'].size(0), -1)

        return super(FLOWT5, self).forward(
            **kwargs
        )

    # We need to resize as B x (N * L) instead of (B * N) x L here
    # because the T5 forward method uses the input tensors to infer
    # dimensions used in the decoder.
    # EncoderWrapper resizes the inputs as (B * N) x L.
    def forward(self, input_ids=None, attention_mask=None, graph=None, **kwargs):
    # 获取 ggnn_output
        ggnn_output = None
        padding_length = 200
        if graph is not None:
            ggnn_output = torch.zeros(input_ids.size(0), padding_length, 768)
            num_nodes = []
            i = 0
            for g in graph:
                
                out = self.flow_gnn(g, {})  # ggnn_output 的形状为 (batch_size,seq, 512)
                if out.size(0)<padding_length:
                    ggnn_output[i, :out.size(0)] = out 
                else:
                    ggnn_output[i, :, :] = out[:padding_length,:] 
                num_nodes.append(out.size(0))
                i += 1
        self.encoder.gnn_out = ggnn_output

        # print(ggnn_output.size())
        # 保证 input_ids 是 2D tensor，确保其类型为 LongTensor
        if input_ids is not None:
            # 如果 input_ids 是 3D 维度，需要展平为 2D
            if input_ids.dim() == 3:
                self.encoder.n_passages = input_ids.size(1)
            input_ids = input_ids.view(input_ids.size(0), -1).long()  # 确保 input_ids 是 LongTensor
            if ggnn_output is not None:
                padding = torch.zeros(input_ids.size(0), padding_length, dtype=input_ids.dtype, device=input_ids.device)  # (batch_size, num_nodes)

        # 在最后一个维度上拼接 input_ids 和 padding
    
                input_ids = torch.cat((input_ids, padding), dim=1)

        if attention_mask is not None:
            attention_mask = attention_mask.view(attention_mask.size(0), -1)  # 确保 attention_mask 也是 2D
            if ggnn_output is not None:
                        # 创建与 input_ids 相同大小的填充
                padding = torch.ones(input_ids.size(0), padding_length, dtype=input_ids.dtype, device=input_ids.device)  # (batch_size, num_nodes)
                for i, num in enumerate(num_nodes):
                    # 将前 num 个元素设置为 1
                    padding[i, :num] = 1
                # 在最后一个维度上拼接 attention_mask 和 padding
                attention_mask = torch.cat((attention_mask, padding), dim=1)

        # 将 ggnn_output 通过 kwargs 传递到 EncoderWrapper 中处理
        # print('input_ids',input_ids.size())
        # print('attention_mask',attention_mask.size())
        return super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask, # 将 ggnn_output 传入到 EncoderWrapper 进行后续处理
            **kwargs,
        )

    # We need to resize the inputs here, as the generate method expect 2D tensors
    def generate(self, input_ids, attention_mask, max_length, graph,**kwargs):
        self.encoder.n_passages = input_ids.size(1)
        ggnn_output = None
        padding_length = 200
        if graph is not None:
            ggnn_output = torch.zeros(input_ids.size(0), padding_length, 768)
            num_nodes = []
            i = 0
            for g in graph:
                
                out = self.flow_gnn(g, {})  # ggnn_output 的形状为 (batch_size,seq, 512)
                if out.size(0)<padding_length:
                    ggnn_output[i, :out.size(0)] = out 
                else:
                    ggnn_output[i, :, :] = out[:padding_length,:] 
                num_nodes.append(out.size(0))
                i += 1
        self.encoder.gnn_out = ggnn_output
        # 保证 input_ids 是 2D tensor，确保其类型为 LongTensor
        if input_ids is not None:
            # 如果 input_ids 是 3D 维度，需要展平为 2D
            if input_ids.dim() == 3:
                self.encoder.n_passages = input_ids.size(1)
            input_ids = input_ids.view(input_ids.size(0), -1).long()  # 确保 input_ids 是 LongTensor
            if ggnn_output is not None:
                padding = torch.zeros(input_ids.size(0), padding_length, dtype=input_ids.dtype, device=input_ids.device)  # (batch_size, num_nodes)

        # 在最后一个维度上拼接 input_ids 和 padding
    
                input_ids = torch.cat((input_ids, padding), dim=1)
        if attention_mask is not None:
            attention_mask = attention_mask.view(attention_mask.size(0), -1)  # 确保 attention_mask 也是 2D
            if ggnn_output is not None:
                        # 创建与 input_ids 相同大小的填充
                padding = torch.ones(input_ids.size(0), padding_length, dtype=input_ids.dtype, device=input_ids.device)  # (batch_size, num_nodes)
                for i, num in enumerate(num_nodes):
                    # 将前 num 个元素设置为 1
                    padding[i, :num] = 1
                # 在最后一个维度上拼接 attention_mask 和 padding
                attention_mask = torch.cat((attention_mask, padding), dim=1)
        
            # 将 ggnn_output 添加到 kwargs 中，以便在编码器中使用
        return super().generate(
            input_ids=input_ids.view(input_ids.size(0), -1),
            attention_mask=attention_mask.view(attention_mask.size(0), -1),
            max_length=max_length,
            **kwargs
        )

    def wrap_encoder(self, use_checkpoint=False):
        """
        Wrap T5 encoder to obtain a Fusion-in-Decoder model.
        """
        self.encoder = EncoderWrapper(self.encoder, use_checkpoint=use_checkpoint)

    def unwrap_encoder(self):
        """
        Unwrap Fusion-in-Decoder encoder, useful to load T5 weights.
        """
        self.encoder = self.encoder.encoder
        block = []
        for mod in self.encoder.block:
            block.append(mod.module)
        block = nn.ModuleList(block)
        self.encoder.block = block

    def load_t5(self, state_dict):
        self.unwrap_encoder()
        self.load_state_dict(state_dict,strict=False)
        self.wrap_encoder()

    def set_checkpoint(self, use_checkpoint):
        """
        Enable or disable checkpointing in the encoder.
        See https://pytorch.org/docs/stable/checkpoint.html
        """
        for mod in self.encoder.encoder.block:
            mod.use_checkpoint = use_checkpoint

    def reset_score_storage(self):
        """
        Reset score storage, only used when cross-attention scores are saved
        to train a retriever.
        """
        for mod in self.decoder.block:
            mod.layer[1].EncDecAttention.score_storage = None

    def get_crossattention_scores(self, context_mask):
        """
        Cross-attention scores are aggregated to obtain a single scalar per
        passage. This scalar can be seen as a similarity score between the
        question and the input passage. It is obtained by averaging the
        cross-attention scores obtained on the first decoded token over heads,
        layers, and tokens of the input passage.

        More details in Distilling Knowledge from Reader to Retriever:
        https://arxiv.org/abs/2012.04584.
        """
        scores = []
        n_passages = context_mask.size(1)
        for mod in self.decoder.block:
            scores.append(mod.layer[1].EncDecAttention.score_storage)
        scores = torch.cat(scores, dim=2)
        bsz, n_heads, n_layers, _ = scores.size()
        # batch_size, n_head, n_layers, n_passages, text_maxlength
        scores = scores.view(bsz, n_heads, n_layers, n_passages, -1)
        scores = scores.masked_fill(~context_mask[:, None, None], 0.)
        scores = scores.sum(dim=[1, 2, 4])
        ntokens = context_mask.sum(dim=[2]) * n_layers * n_heads
        scores = scores/ntokens
        return scores

    def overwrite_forward_crossattention(self):
        """
        Replace cross-attention forward function, only used to save
        cross-attention scores.
        """
        for mod in self.decoder.block:
            attn = mod.layer[1].EncDecAttention
            attn.forward = types.MethodType(cross_attention_forward, attn)

class EncoderWrapper(torch.nn.Module):
    def __init__(self, encoder, use_checkpoint=False):
        super().__init__()
        self.main_input_name = "input_ids"
        # self.linear = nn.Linear(896, 768)  # 用于调整拼接后的 hidden state 大小
        self.encoder = encoder
        self.gnn_out = None
        apply_checkpoint_wrapper(self.encoder, use_checkpoint)

        # Query 和 Key 的线性变换，用于计算注意力权重
        # self.query_layer = nn.Linear(768, 512)  # 用于对T5的hidden state进行投影
        # self.key_layer = nn.Linear(768, 512)    # 用于对GNN的输出进行投影

    def forward(self, input_ids=None, attention_mask=None, ggnn_output=None, **kwargs):
        # print(input_ids.size(),attention_mask.size())
        outputs = self.encoder(input_ids[:,:512], attention_mask[:,:512], **kwargs)
        # print('input_ids', input_ids.size())
        if self.gnn_out is not None:
            self.gnn_out = self.gnn_out.to('cuda:1')
            encoder_hidden_states = outputs.last_hidden_state  # (batch_size, seq_len, hidden_dim)
            # print(encoder_hidden_states.size())
            # ggnn_output 是 (batch_size, 768)，我们需要将其转换为 query，变为 (batch_size, 1, 768)

            encoder_hidden_states = torch.cat((encoder_hidden_states, self.gnn_out), dim=1)  # 替换填充部分
            # print(encoder_hidden_states.size())


            # print('outputs.last_hidden_state',outputs.last_hidden_state)
            outputs.last_hidden_state = encoder_hidden_states

        # print(outputs)
        return outputs
class CheckpointWrapper(torch.nn.Module):
    def __init__(self, module, use_checkpoint=False):
        super().__init__()
        self.module = module
        self.use_checkpoint = use_checkpoint

    def forward(self, hidden_states, attention_mask, position_bias, **kwargs):
        if self.use_checkpoint and self.training:
            kwargs = {k: v for k, v in kwargs.items() if v is not None}
            def custom_forward(*inputs):
                output = self.module(*inputs, **kwargs)
                empty = torch.tensor(
                    [],
                    dtype=torch.float,
                    device=output[0].device,
                    requires_grad=True)
                output = tuple(x if x is not None else empty for x in output)
                return output

            output = torch.utils.checkpoint.checkpoint(
                custom_forward,
                hidden_states,
                attention_mask,
                position_bias
            )
            output = tuple(x if x.size() != 0 else None for x in output)
        else:
            output = self.module(hidden_states, attention_mask, position_bias, **kwargs)
        return output

def apply_checkpoint_wrapper(t5stack, use_checkpoint):
    block = []
    for mod in t5stack.block:
        wrapped_mod = CheckpointWrapper(mod, use_checkpoint)
        block.append(wrapped_mod)
    block = nn.ModuleList(block)
    t5stack.block = block

def cross_attention_forward(
        self,
        input,
        mask=None,
        kv=None,
        position_bias=None,
        past_key_value_state=None,
        head_mask=None,
        query_length=None,
        use_cache=False,
        output_attentions=False,
    ):
    assert(kv != None)
    assert(head_mask == None)
    assert(position_bias != None or self.has_relative_attention_bias)

    bsz, qlen, dim = input.size()
    n_heads, d_heads = self.n_heads, self.d_kv
    klen = kv.size(1)

    q = self.q(input).view(bsz, -1, n_heads, d_heads).transpose(1, 2)
    if past_key_value_state == None:
        k = self.k(kv).view(bsz, -1, n_heads, d_heads).transpose(1, 2)
        v = self.v(kv).view(bsz, -1, n_heads, d_heads).transpose(1, 2)
    else:
        k, v = past_key_value_state

    scores = torch.einsum("bnqd,bnkd->bnqk", q, k)

    if mask is not None:
       scores += mask

    if position_bias is None:
        position_bias = self.compute_bias(qlen, klen)
    scores += position_bias

    if self.score_storage is None:
        self.score_storage = scores

    attn = F.softmax(scores.float(), dim=-1).type_as(scores)
    attn = F.dropout(attn, p=self.dropout, training=self.training)

    output = torch.matmul(attn, v)
    output = output.transpose(1, 2).contiguous().view(bsz, -1, self.inner_dim)
    output = self.o(output)

    if use_cache:
        output = (output,) + ((k, v),)
    else:
        output = (output,) + (None,)

    if output_attentions:
        output = output + (attn,)

    if self.has_relative_attention_bias:
        output = output + (position_bias,)

    return output




import torch
from transformers import AutoTokenizer
import transformers
import dgl
tokenizer = AutoTokenizer.from_pretrained(
    '/home/hdd/qingao/cache/huggingface/transformers/models--Salesforce--codet5-base/snapshots/4078456db09ba972a3532827a0b5df4da172323c'
    )
tokenizer.add_tokens(["Vul_Start","Vul_End"])
model = transformers.T5ForConditionalGeneration.from_pretrained(
    '/home/hdd/qingao/cache/huggingface/transformers/models--Salesforce--codet5-base/snapshots/4078456db09ba972a3532827a0b5df4da172323c'
    )
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load('/home/hdd/qingao/DeepDFA/CodeT5/saved_models/repair/codeT5/checkpoint-best-acc/pytorch_model.bin'))
config = model.config


input_dim = 8
feat = "_ABS_DATAFLOW_datatype_all_limitall_1000_limitsubkeys_1000"
gtype = "cfg"
label_style = "graph"
dsname = "bigvul"
node_type_feat = None
concat_all_absdf = True
hidden_dim = 64
n_steps = 3
num_output_layers = 3

flowgnn_model = FlowGNNGGNNModule(
    feat,
    input_dim,
    hidden_dim,
    n_steps,
    num_output_layers,
    label_style=label_style,
    # freeze_graph=False,
    # append_dataflow="before_graph",
    # codebert_feat=None,
    # doc2vec_feat=None,
    # glove_feat=None,
    # num_node_types=flowgnn_datamodule.num_node_types,
    # node_type_feat=node_type_feat,
    # just_codebert=False,
    concat_all_absdf=concat_all_absdf,
    # undersample_node_on_loss_factor=None,
    # test_every=False,
    # tune_nni=False,
    # positive_weight=None,
    encoder_mode=True,
)


flow_model = FLOWT5(config,flow_gnn=flowgnn_model)
flow_model.load_t5(model.state_dict())

/tmp/ipykernel_2918192/252087155.py:349: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/home/hdd/qingao/DeepDFA/CodeT5/saved_models/repair/

In [18]:
from transformers import AdamW, get_linear_schedule_with_warmup
import os 
# # 冻结 CodeT5 的参数
# for param in flow_model.shared.parameters():
#     param.requires_grad = False

# for param in flow_model.encoder.parameters():
#     param.requires_grad = False

# # 如果 GNN 部分是单独的模块，确保它的参数不被冻结
# # 这里假设 GNN 部分的名称为 `gnn`，请根据实际情况修改
# for param in flow_model.flow_gnn.parameters():
#     param.requires_grad = False  # 确保 GNN 的参数可训练


# for param in flow_model.decoder.parameters():  # 确保流入解码器的参数被冻结
#     param.requires_grad = True

# # 定义学习率和优化器

criterion = nn.CrossEntropyLoss()
lr = 2e-5
num_epochs = 5
best_em = 0
# 冻结 CodeT5 encoder 和 decoder 的参数
output_dir = 'home/hdd/qingao/RMGArepair/saved_model/step3_encoder_decoder_/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for param in flow_model.encoder.parameters():
    param.requires_grad = False

for param in flow_model.decoder.parameters():
    param.requires_grad = False

# 定义不进行权重衰减的参数
no_decay = ['bias', 'LayerNorm.weight']

# 为优化器分组参数
optimizer_grouped_parameters = [
    {'params': [p for n, p in flow_model.named_parameters() if not any(nd in n for nd in no_decay) and p.requires_grad],
     'weight_decay': 0.0
     },
    {'params': [p for n, p in flow_model.named_parameters() if any(nd in n for nd in no_decay) and p.requires_grad], 
     'weight_decay': 0.0}
]

# 定义 AdamW 优化器
optimizer = AdamW(optimizer_grouped_parameters, lr=lr, eps=1e-8)
# for name, param in flow_model.flow_gnn.named_parameters():
#     if param.requires_grad:
#         print(f"Parameter: {name}, Value: {param.data}, Gradient: {param.grad}")
from transformers import get_linear_schedule_with_warmup

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=400,
    num_training_steps=(len(train_dataloader) * num_epochs),
)
# scheduler = get_linear_schedule_with_warmup(optimizer,
#                                             num_warmup_steps=args.warmup_steps,
#                                             num_training_steps=num_train_optimization_steps)


gradient_accumulation_steps = 1
# 训练循环
for cur_epoch in range(0, num_epochs):
    bar = tqdm(train_dataloader, total=len(train_dataloader), desc="Training")
    nb_tr_examples, nb_tr_steps, tr_loss = 0, 0, 0
    flow_model.to('cuda:1')
    flow_model.train()
    
    for step, batch in enumerate(bar):
        input_ids = batch['input_ids'].to('cuda:1')
        attention_mask = batch['attention_mask'].to('cuda:1')
        labels = batch['labels'].to('cuda:1')
        index = batch['index'].to('cuda:1')

        index_list = index.tolist()

        if graphs_by_id is None:
            graphs = None
        else:
            graphs = [graphs_by_id[i].to('cuda:1') for i in index_list if i in graphs_by_id]

            outputs = flow_model(input_ids=input_ids, attention_mask=attention_mask,
                                 labels=labels,
                                 graph=graphs if graphs else None)

        loss = outputs.loss
        tr_loss += loss.item()

        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        loss.backward()

        if nb_tr_steps % gradient_accumulation_steps == 0:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            # print("Flow GNN Parameters and Gradients:")
            # for name, param in flow_model.flow_gnn.named_parameters():
            #     if param.requires_grad:
            #         print(f"Parameter: {name}, Value: {param.data}, Gradient: {param.grad}")
            # break
            # 计算和打印平均损失
            train_loss = round(tr_loss / nb_tr_steps, 4) if nb_tr_steps > 0 else 0
            bar.set_description("[{}] Train loss {}".format(cur_epoch, round(train_loss, 3)))
    em = eval(flow_model,eval_dataloader,graphs_by_id)
    print(em)
    if em>best_em:
        best_em = em
        output_model_file = os.path.join(output_dir, "pytorch_model.bin")
        torch.save(flow_model.state_dict(), output_model_file)
        print("Save the best acc model into %s", output_model_file)


for param in flow_model.encoder.parameters():
    param.requires_grad = True

for param in flow_model.decoder.parameters():
    param.requires_grad = True

# 定义不进行权重衰减的参数
no_decay = ['bias', 'LayerNorm.weight']

# 为优化器分组参数
optimizer_grouped_parameters = [
    {'params': [p for n, p in flow_model.named_parameters() if not any(nd in n for nd in no_decay) and p.requires_grad],
     'weight_decay': 0.0},
    {'params': [p for n, p in flow_model.named_parameters() if any(nd in n for nd in no_decay) and p.requires_grad], 
     'weight_decay': 0.0}
]

# 定义 AdamW 优化器
optimizer = AdamW(optimizer_grouped_parameters, lr=lr, eps=1e-8)
# for name, param in flow_model.flow_gnn.named_parameters():
#     if param.requires_grad:
#         print(f"Parameter: {name}, Value: {param.data}, Gradient: {param.grad}")
from transformers import get_linear_schedule_with_warmup

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=400,
    num_training_steps=(len(train_dataloader) * num_epochs),
)
# scheduler = get_linear_schedule_with_warmup(optimizer,
#                                             num_warmup_steps=args.warmup_steps,
#                                             num_training_steps=num_train_optimization_steps)


gradient_accumulation_steps = 1
# 训练循环
for cur_epoch in range(0, 10):
    bar = tqdm(train_dataloader, total=len(train_dataloader), desc="Training")
    nb_tr_examples, nb_tr_steps, tr_loss = 0, 0, 0
    flow_model.to('cuda:1')
    flow_model.train()
    
    for step, batch in enumerate(bar):
        input_ids = batch['input_ids'].to('cuda:1')
        attention_mask = batch['attention_mask'].to('cuda:1')
        labels = batch['labels'].to('cuda:1')
        index = batch['index'].to('cuda:1')

        index_list = index.tolist()

        if graphs_by_id is None:
            graphs = None
        else:
            graphs = [graphs_by_id[i].to('cuda:1') for i in index_list if i in graphs_by_id]

            outputs = flow_model(input_ids=input_ids, attention_mask=attention_mask,
                                 labels=labels,
                                 graph=graphs if graphs else None)
            


        loss = outputs.loss
        tr_loss += loss.item()

        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        loss.backward()

        if nb_tr_steps % gradient_accumulation_steps == 0:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            # print("Flow GNN Parameters and Gradients:")
            # for name, param in flow_model.flow_gnn.named_parameters():
            #     if param.requires_grad:
            #         print(f"Parameter: {name}, Value: {param.data}, Gradient: {param.grad}")
            # break
            # 计算和打印平均损失
            train_loss = round(tr_loss / nb_tr_steps, 4) if nb_tr_steps > 0 else 0
            bar.set_description("[{}] Train loss {}".format(cur_epoch, round(train_loss, 3)))
    em = eval(flow_model,eval_dataloader,graphs_by_id)
    print(em)
    if em>best_em:
        best_em = em
        output_model_file = os.path.join(output_dir, "pytorch_model.bin")
        torch.save(flow_model.state_dict(), output_model_file)
        print("Save the best acc model into %s", output_model_file)



Training:   0%|          | 0/410 [00:00<?, ?it/s]

[0] Train loss 0.087:  38%|███▊      | 154/410 [02:05<03:28,  1.23it/s]


KeyboardInterrupt: 

In [17]:
import os 
wandb_project = "t5_gnn_tuning"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project

In [18]:
!pip install transformers[torch]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: pytorch-lightning 1.7.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
!pip uninstall keras
!pip install tf-keras


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: keras 3.5.0
Uninstalling keras-3.5.0:
  Would remove:
    /root/miniconda3/envs/deepdfa2/lib/python3.10/site-packages/keras-3.5.0.dist-info/*
    /root/miniconda3/envs/deepdfa2/lib/python3.10/site-packages/keras/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: pytorch-lightning 1.7.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


ValueError: Caught ValueError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/root/miniconda3/envs/deepdfa2/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 84, in _worker
    output = module(*input, **kwargs)
  File "/root/miniconda3/envs/deepdfa2/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/root/miniconda3/envs/deepdfa2/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1562, in _call_impl
    return forward_call(*args, **kwargs)
  File "/tmp/ipykernel_1716557/3591218435.py", line 48, in forward
    return super().forward(
  File "/home/l1/qingao/DeepDFA/CodeT5/modeling_t5.py", line 1752, in forward
    decoder_outputs = self.decoder(
  File "/root/miniconda3/envs/deepdfa2/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/root/mi